In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('returns_sustainability_dataset.csv')
df.head()

# We loaded the dataset to our notebook and displayed the first five rows

,Order_ID,Product_ID,User_ID,Order_Date,Product_Category,Product_Price,Order_Quantity,Discount_Applied,Shipping_Method,Payment_Method,...,Return_Status,Return_Reason,Days_to_Return,Order_Value,Return_Cost,Profit_Loss,CO2_Emissions,Packaging_Waste,CO2_Saved,Waste_Avoided
0,ORD00000,PROD0169,USER0195,2022-01-14,Clothing,1720.71,2,30.46,Next-Day,Wallet,...,Not Returned,No Return,0,2393.163468,0,2393.163468,2.0,0.4,2.0,0.4
1,ORD00001,PROD0318,USER1469,2022-01-03,Toys,744.06,5,29.62,Next-Day,Wallet,...,Returned,Size Issue,12,2618.347140,200,2418.347140,2.0,1.0,0.0,0.0
2,ORD00002,PROD0427,USER1812,2025-03-16,Clothing,983.68,5,47.80,Express,Wallet,...,Not Returned,No Return,0,2567.404800,0,2567.404800,1.5,1.0,1.5,1.0
3,ORD00003,PROD0323,USER1274,2024-11-06,Books,1855.65,2,2.90,Express,COD,...,Not Returned,No Return,0,3603.672300,0,3603.672300,1.5,0.4,1.5,0.4
4,ORD00004,PROD0325,USER0551,2023-06-07,Home Appliances,1770.97,5,44.42,Express,COD,...,Returned,Size Issue,11,4921.525630,200,4721.525630,1.5,1.0,0.0,0.0


In [3]:
df.shape
# We can understand that we have 5000 rows and 23 columns in the dataset

(5000, 23)

In [4]:
df.columns = df.columns.str.lower()
df.columns

# Column names were in Sentence case, so converted them to lower case. This is much easier while working.

Index(['order_id', 'product_id', 'user_id', 'order_date', 'product_category',
       'product_price', 'order_quantity', 'discount_applied',
       'shipping_method', 'payment_method', 'user_age', 'user_gender',
       'user_location', 'return_status', 'return_reason', 'days_to_return',
       'order_value', 'return_cost', 'profit_loss', 'co2_emissions',
       'packaging_waste', 'co2_saved', 'waste_avoided'],
      dtype='object')

In [5]:
df.dtypes
# Checking the data type of the content present in each column

order_id             object
product_id           object
user_id              object
order_date           object
product_category     object
product_price       float64
order_quantity        int64
discount_applied    float64
shipping_method      object
payment_method       object
user_age              int64
user_gender          object
user_location        object
return_status        object
return_reason        object
days_to_return        int64
order_value         float64
return_cost           int64
profit_loss         float64
co2_emissions       float64
packaging_waste     float64
co2_saved           float64
waste_avoided       float64
dtype: object

In [6]:
df.isnull().sum()
# We are checking, whether there are any null values in the datasets by using 'isnull()'
# Using '.sum()' gives us total number of null values in the dataset by adding the count.

order_id            0
product_id          0
user_id             0
order_date          0
product_category    0
product_price       0
order_quantity      0
discount_applied    0
shipping_method     0
payment_method      0
user_age            0
user_gender         0
user_location       0
return_status       0
return_reason       0
days_to_return      0
order_value         0
return_cost         0
profit_loss         0
co2_emissions       0
packaging_waste     0
co2_saved           0
waste_avoided       0
dtype: int64

In [7]:
df['returned'] = (
    df['return_status']
    .astype(str)
    .str.lower()
    .eq('returned')
)

# The `Returned` column is derived from `Return_Status` to create a consistent boolean flag for 
# KPI calculations and downstream BI usage.

In [8]:
# Adding time features (Years, Months)
df['order_date'] = pd.to_datetime(df['order_date'], errors = 'coerce')
df['order_month'] = df['order_date'].dt.to_period('M').astype(str)
df['order_year'] = df['order_date'].dt.to_period('Y').astype(str)

In [9]:
df[['order_date', 'order_month', 'order_year']].head()

,order_date,order_month,order_year
0,2022-01-14,2022-01,2022
1,2022-01-03,2022-01,2022
2,2025-03-16,2025-03,2025
3,2024-11-06,2024-11,2024
4,2023-06-07,2023-06,2023


In [10]:
# Performing Consistency Check
df[(df['returned'] == False) & (df['return_cost'] > 0)].shape[0]

# We wrote this line to check whether the product is NOT RETURNED and there is no return cost for that.

0

In [11]:
# Core KPI
# Answering: How big is the returning problem?
# Answering: What is the financial Impact?

total_orders = len(df) # Total number of orders calculated
returned_orders = df['returned'].sum() # Getting the number of orders where the product is returned
return_rate = (returned_orders/total_orders) * 100 # Percentage of return rate

total_return_cost = df.loc[df['returned'], 'return_cost'].sum()
# We are using .loc to determine the value from return_cost column, only if the returned column brings up "returned"
avg_return_cost = df.loc[df['returned'], 'return_cost'].mean()
# Similar to the above line we are finding the average, by using df.loc.

print('Total Orders:',total_orders)
print('Total number of orders returned:',returned_orders)
print('Rate of return:',round(return_rate,2),'%')
print('Total return cost:',total_return_cost)
print('Average return cost:',avg_return_cost)

Total Orders: 5000
Total number of orders returned: 1450
Rate of return: 29.0 %
Total return cost: 290000
Average return cost: 200.0


In [12]:
# Returns reason analysis
reason_counts = df.loc[df['returned'], 'return_reason'].value_counts()
print(reason_counts)
# Doing the above step gives up the count of reasons for the products being returned

return_reason
Defective       382
Changed Mind    379
Wrong Item      348
Size Issue      341
Name: count, dtype: int64


In [13]:
# Category-level analysis
cat_summary = df.loc[df['returned'], 'product_category'].value_counts()
cat_summary


product_category
Clothing           661
Electronics        314
Toys               181
Books              172
Home Appliances    122
Name: count, dtype: int64

In [40]:
# Product Category-level anaylsis
cat_summary = df.groupby("product_category").agg(
    orders=("returned", "size"),
    returns=("returned", "sum"),
    return_rate=("returned", "mean")
)

In [77]:
# Location wise analysis (Where returns are more)
df[df['returned']].groupby('user_location').size()

user_location
City1      14
City10     14
City100    14
City11     14
City12     17
           ..
City95     14
City96     14
City97     14
City98     13
City99     13
Length: 100, dtype: int64

In [85]:
# Time trend (when returns are high)
monthly = df.groupby(
    pd.Grouper(key = 'order_date', freq = 'ME'))['returned'].mean()
monthly

order_date
2022-01-31    0.272727
2022-02-28    0.288288
2022-03-31    0.215686
2022-04-30    0.288136
2022-05-31    0.256198
2022-06-30    0.233871
2022-07-31    0.297521
2022-08-31    0.285714
2022-09-30    0.263636
2022-10-31    0.271845
2022-11-30    0.235294
2022-12-31    0.264000
2023-01-31    0.220183
2023-02-28    0.262626
2023-03-31    0.358974
2023-04-30    0.363636
2023-05-31    0.278689
2023-06-30    0.260000
2023-07-31    0.308511
2023-08-31    0.258065
2023-09-30    0.300885
2023-10-31    0.350427
2023-11-30    0.302752
2023-12-31    0.293103
2024-01-31    0.294643
2024-02-29    0.338583
2024-03-31    0.296296
2024-04-30    0.269565
2024-05-31    0.381818
2024-06-30    0.271028
2024-07-31    0.352459
2024-08-31    0.284553
2024-09-30    0.358696
2024-10-31    0.342857
2024-11-30    0.280374
2024-12-31    0.300000
2025-01-31    0.236842
2025-02-28    0.367347
2025-03-31    0.248062
2025-04-30    0.339623
2025-05-31    0.234375
2025-06-30    0.258929
2025-07-31    0.346774


In [88]:
# Operational Drivers (How business decision affect)
shipping_returns = df.groupby('shipping_method')['returned'].mean()
payment_returns = df.groupby('payment_method')['returned'].mean()

In [105]:
# Discount Analysis
df['discount_bin'] = pd.qcut(df["discount_applied"], 4)

In [102]:
print(f"Overall return rate: {return_rate:.2%}")


Overall return rate: 2900.00%


In [108]:
df.to_csv('returns_cleaned_for_sql_powerbi.csv', index = False)